In [40]:
from transformers import AutoTokenizer
import torch.nn as nn
from torch.utils.data import DataLoader
import torch

tokenizer=AutoTokenizer.from_pretrained('gpt2')
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [154]:
print(tokenizer.pad_token)

None


In [4]:
from datasets import load_dataset

ds = load_dataset("CohleM/english-to-nepali")

In [5]:
ds['train']

Dataset({
    features: ['en', 'ne'],
    num_rows: 177334
})

In [8]:
train_dataloader=DataLoader(ds['train'],batch_size=64,shuffle=True)
train_dataloader

In [185]:
# dictionary for the tokenized data
t_dict={'en':[],
        'ne':[]
        }


In [132]:
# making a dictionary of tokens for both english and nepali language

# extract every sentence from the dataloader and tokenize it one by one.. not a good method. need to add padding manually by
# deriving the maximum length of tokens for each batch and after that add padding to make the same length.
# padding is needed beacuse we are making the batch. otherwise it is not needed in RNN.

# for data in train_dataloader:

#     for sentence in data['en']:
#         t_dict['en'].append(tokenizer.encode(sentence))
    
#     for sentence in data['ne']:
#         t_dict['ne'].append(tokenizer.encode(sentence))
    
#     break


In [186]:
# gpt2 doesnt have special tokens to adding a padding token
tokenizer.add_special_tokens({'pad_token':'<pad>'})

0

In [187]:
for data in train_dataloader:
    t_dict['en']=tokenizer(data['en'],padding=True)['input_ids']
    t_dict['ne']=tokenizer(data['ne'],padding=True)['input_ids']
    print(t_dict['en'])
    break

[[1639, 481, 423, 6088, 284, 4483, 11, 290, 307, 11378, 11, 290, 481, 13463, 262, 1438, 286, 35473, 732, 71, 11, 534, 1793, 11, 508, 468, 11829, 3135, 7596, 306, 351, 345, 26, 290, 616, 661, 481, 1239, 757, 307, 11679, 13, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], [2215, 340, 373, 1297, 31603, 11, 339, 1908, 584, 2085, 9302, 11, 290, 484, 635, 48897, 798, 13, 31603, 1908, 2085, 9302, 757, 262, 2368, 640, 11, 290, 484, 635, 48897, 798, 13, 50257, 50257, 50257, 50257, 50257, 502

In [135]:

# # finding maximum sentence length for each batch for english langauge
# sen_length=[]
# for sen in t_dict['en']:
#     sen_length.append(len(sen))

# # add zero padding to all token if not of same length(max length)
# for sen in t_dict['en']:
#     for i in range(len(sen),max(sen_length)):
#         sen.append(0)


In [136]:
# # finding maximum sentence length
# max(sen_length)

86

In [139]:
embed=nn.Embedding(100000,128,padding_idx=0)
input=torch.IntTensor(t_dict['en'][0]).unsqueeze(0)
out=embed(input)

In [140]:
out.shape

torch.Size([1, 86, 128])

In [181]:
class rnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding=nn.Embedding(50258,128,padding_idx=50257)
        self.rnn=nn.RNN(128,hidden_size=32,batch_first=True)

    def forward(self,x,hidden=None):
        x=self.embedding(x)
        x,hidden=self.rnn(x,hidden)
        return x,hidden



In [182]:
model_rnn=rnn()
model_rnn

rnn(
  (embedding): Embedding(50258, 128, padding_idx=50257)
  (rnn): RNN(128, 32, batch_first=True)
)

In [183]:
x,y=model_rnn(torch.tensor(t_dict['en']))

In [184]:
x.shape,y.shape

(torch.Size([64, 111, 32]), torch.Size([1, 64, 32]))